In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Is this notebook running on Colab or Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from typing import Dict, Text
from tensorflow import keras
import tensorflow_recommenders as tfrs
try:
    if not tf.config.list_physical_devices('GPU'):
        assert tf.__version__ >= "2.0"
        print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
        if IS_COLAB:
            print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
        if IS_KAGGLE:
            print("Go to Settings > Accelerator and select GPU.")
except:
    if not tf.test.is_gpu_available():
        assert tf.__version__ >= "2.0"
        print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
        if IS_COLAB:
            print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
        if IS_KAGGLE:
            print("Go to Settings > Accelerator and select GPU.")

# Common imports
import numpy as np
import pandas as pd
import sklearn.preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import os
import datetime as dt
from pathlib import Path

# to make this notebook's output stable across runs
np.random.seed(42)
try:
    if not tf.config.list_physical_devices('GPU'):
        tf.random.set_seed(42)
    else:
        tf.random.set_random_seed(42)
except:
    if not tf.test.is_gpu_available():
        tf.random.set_seed(42)
    else:
        tf.random.set_random_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

No GPU was detected. LSTMs and CNNs can be very slow without a GPU.


In [2]:
def read_files(file_path, **kwargs):
    
    art_df_args = dict(filepath_or_buffer=file_path + 'articles.csv',low_memory = False)
    if 'art_cols' in kwargs:
        art_df_args['usecols']=kwargs['art_cols']
    
    cust_df_args = dict(filepath_or_buffer=file_path + 'customers.csv', low_memory = False)
    if  'cust_cols' in  kwargs:
        cust_df_args['usecols']=kwargs['cust_cols']
    
    trans_df_args= dict(filepath_or_buffer=file_path + 'transactions_train.csv', low_memory = False)
    if  'trans_cols' in kwargs:
        trans_df_args['usecols']=kwargs['trans_cols']
    
    art_df = pd.read_csv(**art_df_args)
    cust_df = pd.read_csv(**cust_df_args)
    trans_df= pd.read_csv(**trans_df_args)
    
    customer_lookup = cust_df.reset_index().set_index('customer_id')['index'].astype(str).to_dict()
    article_lookup =art_df.reset_index().set_index('article_id')['index'].astype(str).to_dict()
    
    trans_df['user_id']= trans_df['customer_id'].map(customer_lookup)
    trans_df['item_id']= trans_df['article_id'].map(article_lookup)
    
    unique_users = trans_df['user_id'].unique()
    unique_items = trans_df['item_id'].unique()
    
    trans_df = trans_df.drop(columns =['customer_id','article_id'])
    
    return customer_lookup, article_lookup, trans_df, unique_users, unique_items


In [ ]:
file_path = 'h-and-m-personalized-fashion-recommendations/'
customer_lookup, article_lookup, trans_data, user_vocab, item_vocab = read_files(file_path, cust_cols=['customer_id'], trans_cols= ['customer_id','article_id'])
train_size =0.80
np.random.seed(1221)
train = trans_data[['user_id','item_id']].sample(frac=train_size)
temp =  trans_data[['user_id','item_id']].drop(train.index)
valid=temp[['user_id','item_id']].sample(frac=0.5)
test=temp[['user_id','item_id']].drop(valid.index)

In [ ]:
train.tail()

In [ ]:
test.head()

In [ ]:
embedding_dimension = 32

In [ ]:
user_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
        vocabulary = user_vocab, mask_token =None),
    tf.keras.layers.Embedding(len(user_vocab)+1, embedding_dimension)])

In [ ]:
item_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
        vocabulary = item_vocab, mask_token =None),
    tf.keras.layers.Embedding(len(item_vocab)+1, embedding_dimension)
])

In [ ]:
metrics = tfrs.metrics.FactorizedTopK(
    candidates=items.batch(256).map(item_model))

task = tfrs.tasks.Retrieval(metrics=metrics)

In [ ]:
class UserItemModel(tfrs.Model):
    
    def __init__(self, user_model, item_model):
        super().__init__()
        self.user_model : tf.keras.Model = user_model
        self.item_model : tf.keras.Model = item_model
        self.task : tf.keras.layers.Layer = task
            
    def compute_loss(self,features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        
        user_embeddings = self.user_model(features['user_id'])
        positive_item_embeddings = self.item_model(features['item_id'])
        
        return self.task(user_embeddings,positive_item_embeddings)

In [ ]:
model = UserItemModel(user_model, item_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [ ]:
cached_train = train.batch(8192).cache()
cached_valid = valid.batch(4096).cache()
cached_test = test.batch(4096).cache()

In [ ]:
model.fit(cached_train, epochs=10,validation_data=cached_valid)